In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load historical stock price data from CSV
data = pd.read_csv("historical_data.csv.csv")

# Preprocess data
data = data.drop("Date", axis=1)  # Remove the Date column
data = data.values  # Convert DataFrame to NumPy array

# Split data into training and testing sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size, :], data[train_size:len(data), :]

# Prepare data for LSTM input
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), :])
        Y.append(data[i + look_back, 0])  # Predict the first column (Close price)
    return np.array(X), np.array(Y)




In [7]:
look_back = 10
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

print(X_train.shape)
print(X_test.shape)




(195, 1, 6)
(48, 1, 6)


In [8]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, 6)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=2)



Epoch 1/100

7/7 - 2s - loss: 2133737.2500 - 2s/epoch - 265ms/step
Epoch 2/100
7/7 - 0s - loss: 2133434.0000 - 17ms/epoch - 2ms/step
Epoch 3/100
7/7 - 0s - loss: 2133133.2500 - 14ms/epoch - 2ms/step
Epoch 4/100
7/7 - 0s - loss: 2132834.7500 - 15ms/epoch - 2ms/step
Epoch 5/100
7/7 - 0s - loss: 2132537.5000 - 13ms/epoch - 2ms/step
Epoch 6/100
7/7 - 0s - loss: 2132238.7500 - 12ms/epoch - 2ms/step
Epoch 7/100
7/7 - 0s - loss: 2131940.2500 - 14ms/epoch - 2ms/step
Epoch 8/100
7/7 - 0s - loss: 2131641.5000 - 15ms/epoch - 2ms/step
Epoch 9/100
7/7 - 0s - loss: 2131344.0000 - 16ms/epoch - 2ms/step
Epoch 10/100
7/7 - 0s - loss: 2131045.5000 - 15ms/epoch - 2ms/step
Epoch 11/100
7/7 - 0s - loss: 2130747.0000 - 13ms/epoch - 2ms/step
Epoch 12/100
7/7 - 0s - loss: 2130447.5000 - 13ms/epoch - 2ms/step
Epoch 13/100
7/7 - 0s - loss: 2130149.2500 - 14ms/epoch - 2ms/step
Epoch 14/100
7/7 - 0s - loss: 2129850.7500 - 15ms/epoch - 2ms/step
Epoch 15/100
7/7 - 0s - loss: 2129553.0000 - 15ms/epoch - 2ms/step
E

In [10]:

# Make predictions on the test set
predictions = model.predict(X_test)



2/2 [==============================] - 0s 3ms/step


In [11]:
# Evaluate model performance (e.g., using RMSE)
rmse = np.sqrt(np.mean(predictions - Y_test) ** 2)
print("RMSE:", rmse)



RMSE: 1641.9639879481608


In [12]:
# Get current price and predict next price
current_price = data[-1, 0]  # Assuming data is sorted by date



In [15]:
current_price_data = data[-look_back:, :]  # Get the last look_back rows of data
print(current_price_data)


[[  1685.         1685.         1658.650024   1671.849976   1671.849976
  606862.      ]]


In [16]:
current_price_reshaped = np.reshape(current_price_data, (1, 1, look_back * X_train.shape[2]))
print(current_price_reshaped)

[[[  1685.         1685.         1658.650024   1671.849976   1671.849976
   606862.      ]]]


In [17]:
next_price = model.predict(current_price_reshaped)
print("Predicted next price:", next_price[0][0])

1/1 [==============================] - 0s 18ms/step
Predicted next price: 10.479763
